In [76]:
import os
os.environ["OMP_NUM_THREADS"] = '1'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from sklearn.metrics import pairwise_distances
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

from scipy.signal import find_peaks
from kneed import KneeLocator

import ast

In [77]:
df = pd.read_csv('5LE5_residues_dataframe.txt')
convert_dic={'A':'A1', 'B':'A2', 'C':'A3', 'D':'A4', 'E':'A5', 'F':'A6', 'G':'A7', 'H':'H1', 'I':'H2', 'J':'H3', 'K':'H4', 'L':'H5', 'a':'H6', 'b':'H7'}
df['res1'] = df['res1'].apply(ast.literal_eval)
df['res2'] = df['res2'].apply(ast.literal_eval)
df['pair'] = df['chain1']+'_'+df['chain2']+'_'+df['rank'].astype(str)
#df[df['pair'].str.contains('A')]
df

,chain1,chain2,rank,res1,res2,pair
0,a,a,0,"[2, 3, 4, 5, 7, 9, 29, 30, 32, 35, 100, 103, 1...","[2, 3, 4, 5, 7, 9, 29, 30, 32, 35, 100, 103, 1...",a_a_0
1,A,a,0,"[9, 11, 13, 18, 19, 20, 21, 22, 23, 25, 26, 29...","[1, 2, 3, 4, 5, 6, 30, 32, 33, 35, 36, 39, 56,...",A_a_0
2,A,A,0,"[2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...","[1, 2, 3, 4, 6, 7, 8, 10, 11, 20, 21, 23, 24, ...",A_A_0
3,a,a,1,"[3, 4, 7, 9, 10, 32, 33, 34, 100, 107, 109, 12...","[3, 4, 7, 9, 10, 32, 33, 100, 107, 109, 126, 1...",a_a_1
4,A,a,1,"[7, 9, 11, 12, 13, 18, 19, 20, 21, 22, 23, 26,...","[1, 2, 3, 4, 5, 6, 30, 32, 35, 36, 39, 56, 57,...",A_a_1
...,...,...,...,...,...,...
520,L,L,0,"[1, 3, 4, 5, 6, 7, 30, 33, 34, 35, 97, 100, 10...","[1, 3, 4, 5, 6, 7, 30, 33, 34, 35, 97, 100, 10...",L_L_0
521,L,L,1,"[1, 2, 3, 4, 5, 7, 33, 93, 97, 100, 101, 102, ...","[1, 2, 3, 4, 5, 7, 33, 93, 97, 100, 101, 102, ...",L_L_1
522,L,L,2,"[1, 2, 3, 4, 5, 6, 7, 33, 93, 101, 102, 103, 1...","[1, 2, 3, 4, 5, 6, 7, 33, 93, 101, 102, 103, 1...",L_L_2
523,L,L,3,"[1, 2, 3, 4, 5, 7, 30, 33, 34, 100, 101, 102, ...","[1, 2, 3, 4, 5, 7, 30, 33, 34, 100, 101, 102, ...",L_L_3


In [71]:
opt_clusters={}

for chain in 'ABCDEFGHIJKLab':
    df_chain=df[df['pair'].str.contains(chain)]

    # residue vector
    residues = []
    for i, row in df_chain.iterrows():
        if row['chain1']==chain:
            residues.extend(row['res1'])
        elif row['chain2']==chain:
            residues.extend(row['res2'])
    residues = list(set(residues))

    # make dataframe
    columns=[]
    M=np.zeros((len(residues),len(df_chain['chain2'])))
    cn=0
    for i,row in df_chain.iterrows():
        if row['chain1']==chain:
            columns.append(convert_dic[row['pair'][2]]+'_'+row['pair'][4])
            for res in row['res1']:
                rn = residues.index(res)
                M[rn,cn]+=1
            cn+=1
        elif row['chain2']==chain:
            columns.append(convert_dic[row['pair'][0]]+'_'+row['pair'][4])
            for res in row['res2']:
                rn = residues.index(res)
                M[rn,cn]+=1
            cn+=1
    M=pd.DataFrame(M, index=residues, columns= columns)
    #sns.clustermap(M.T,figsize=(10, 10))
    #plt.title(chain)
    #plt.show()

    wcss = []
    for i in range(1, 8):
        kmeans = KMeans(n_clusters=i, init='random', random_state=25,n_init='auto') #k-means++ #best one --> random,random_state:25, range:8, n_init='auto
        kmeans.fit(M.T.to_numpy())
        wcss.append(kmeans.inertia_)
    # Find the knee point using the KneeLocator module
    kl = KneeLocator(range(1, 8), wcss, curve='convex', direction='decreasing')
    knee = kl.elbow
    opt_clusters[chain]=kl.elbow

    #plt.plot(range(1, 8), wcss)
    #plt.xlabel('Number of Clusters')
    #plt.ylabel('WCSS')
    #plt.title('Elbow Method')
    #plt.vlines(knee, plt.ylim()[0], plt.ylim()[1], linestyles='dashed')
    #plt.show()
opt_clusters

{'A': 3,
 'B': 3,
 'C': 3,
 'D': 2,
 'E': 2,
 'F': 3,
 'G': 3,
 'H': 4,
 'I': 4,
 'J': 3,
 'K': 4,
 'L': 3,
 'a': 3,
 'b': 4}

In [78]:
groups_dic={}
for chain in 'ABCDEFGHIJKLab':
    df_chain=df[df['pair'].str.contains(chain)]

    # residue vector
    residues = []
    for i, row in df_chain.iterrows():
        if row['chain1']==chain:
            residues.extend(row['res1'])
        elif row['chain2']==chain:
            residues.extend(row['res2'])
    residues = list(set(residues))

    # make dataframe
    columns=[]
    M=np.zeros((len(residues),len(df_chain['chain2'])))
    cn=0
    for i,row in df_chain.iterrows():
        if row['chain1']==chain:
            columns.append(convert_dic[row['pair'][2]]+'_'+row['pair'][4])
            for res in row['res1']:
                rn = residues.index(res)
                M[rn,cn]+=1
            cn+=1
        elif row['chain2']==chain:
            columns.append(convert_dic[row['pair'][0]]+'_'+row['pair'][4])
            for res in row['res2']:
                rn = residues.index(res)
                M[rn,cn]+=1
            cn+=1
    M=pd.DataFrame(M, index=residues, columns= columns).T

    groups_dic[chain]=[[] for i in range(opt_clusters[chain])]
    # Create the KMeans object with 3 clusters
    kmeans = KMeans(n_clusters=opt_clusters[chain],n_init='auto')

    # Fit the model to the dataset
    kmeans.fit(M)

    # Get the cluster labels for each data point
    labels = kmeans.labels_

    # Print the cluster labels for each row
    for i, label in enumerate(labels):
        #print(f"row {i+1}: Cluster {label+1}")
        #labels_dic[M.index[i]]=label+1
        groups_dic[chain][int(label)].append(M.index[i])
        
#groups_dic['A'][2]

In [79]:
groups_dic

{'A': [['A1_1',
   'A1_2',
   'A1_3',
   'A1_4',
   'A2_0',
   'A2_1',
   'A3_0',
   'A3_1',
   'A3_3',
   'A5_0',
   'A5_1',
   'A5_2',
   'A5_3',
   'A5_4',
   'A6_0',
   'A7_0',
   'A7_1',
   'A7_2',
   'A7_3',
   'A7_4'],
  ['H6_2',
   'H6_4',
   'H7_0',
   'H7_1',
   'H7_2',
   'H7_3',
   'H7_4',
   'H1_0',
   'H1_1',
   'H1_2',
   'H1_3',
   'H1_4',
   'H2_0',
   'H2_1',
   'H2_2',
   'H2_3',
   'H2_4',
   'H3_0',
   'H3_1',
   'H3_2',
   'H3_3',
   'H3_4',
   'H4_0',
   'H4_1',
   'H4_2',
   'H4_3',
   'H4_4',
   'H5_0',
   'H5_1',
   'H5_2',
   'H5_3',
   'H5_4'],
  ['H6_0',
   'A1_0',
   'H6_1',
   'H6_3',
   'A2_2',
   'A2_3',
   'A2_4',
   'A3_2',
   'A3_4',
   'A4_0',
   'A4_1',
   'A4_2',
   'A4_3',
   'A4_4',
   'A6_1',
   'A6_2',
   'A6_3',
   'A6_4']],
 'B': [['H6_0',
   'H6_1',
   'H6_2',
   'H6_3',
   'H6_4',
   'H7_0',
   'H7_1',
   'H7_2',
   'H7_3',
   'H7_4',
   'H1_0',
   'H1_1',
   'H1_2',
   'H1_3',
   'H1_4',
   'H2_0',
   'H2_1',
   'H2_2',
   'H2_3',
   'H2_

In [10]:
import pickle 
model = pickle.load(open('final_regression.sav', 'rb'))

Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-06-05 17:18:10         2564
metadata.json                                  2023-06-05 17:18:10           64
variables.h5                                   2023-06-05 17:18:10       157960
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...layers\dropout
......vars
...layers\dropout_1
......vars
...metrics\mean
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars


In [74]:
df = pd.read_csv('5LE5_final_scores.txt')

selected_features = ['IntraclashesGroup1', 'IntraclashesGroup2', 'Interaction Energy',
       'Backbone Hbond', 'Sidechain Hbond', 'Van der Waals', 'Electrostatics',
       'Solvation Polar', 'Solvation Hydrophobic', 'Van der Waals clashes',
       'entropy sidechain', 'entropy mainchain', 'ptm_iptm',
       'pdockq2', 'pdockq1', 'backbone clash',
       'Interface Residues', 'pred_contact', 'Interface Residues Clashing', 'electrostatic kon',
       'Interface Residues BB Clashing', 'Interface Residues VdW Clashing'] #'Number of Residues'

def just_minmaxscale(df, feature):
    ### just scale within each structure
    #if feature in neg_cor_features: df[feature] = df[feature] * (-1)
    minimum = df[feature].min()
    maximum = df[feature].max()
    SD = df[feature].std()
    Mean = df[feature].mean()
    n = len(set(df['chain1']).union(set(df['chain2'])))
    if maximum==minimum:
        df[feature] = df[feature]-df[feature]+0.5
    else:
        df[feature] = (df[feature]-minimum)/(maximum-minimum)
    return df

def ranked_minmaxscale(df, feature):
    ### just scale within each structure
    #if feature in neg_cor_features: df[feature] = df[feature] * (-1)
    df[feature] = df[feature].rank()
    minimum = df[feature].min()
    maximum = df[feature].max()
    SD = df[feature].std()
    Mean = df[feature].mean()
    n = len(set(df['chain1']).union(set(df['chain2'])))
    if maximum==minimum:
        df[feature] = df[feature]
    else:
        df[feature] = (df[feature]-minimum)/(maximum-minimum)
    return df

# prepare the modified Data
df2= pd.DataFrame()
df2['pair']=df['pdb_id']
df2['pdb_id'] = df2['pair'].apply(lambda x: x.split('_')[0])
df2['chain1'] = df2['pair'].apply(lambda x: x.split('_')[1])
df2['chain2'] = df2['pair'].apply(lambda x: x.split('_')[3])

df2[selected_features]=df[selected_features]

for feature in selected_features:
    if feature in ['pred_contact','pdockq2', 'pdockq1','ptm_iptm']:
        df2 = df2.groupby('pdb_id').apply(lambda x: ranked_minmaxscale(x, feature))
    #elif feature in ['obs_contact']:
     #   df2 = df2.groupby('pdb_id').apply(lambda x: just_minmaxscale(x, feature))
    else:
        df2 = df2.groupby('pdb_id').apply(lambda x: ranked_minmaxscale(x, feature))

X_test = df2.drop(['pdb_id','chain1','chain2', 'pair'], axis=1)

predictions={}
y_pred = model.predict(X_test)
for i in range(len(df['pdb_id'])):
    predictions[df['pdb_id'][i]]=float(y_pred[i])
    
predictions

17/17 [==============================] - 0s 1ms/step


{'5LE5_C_5LE5_G_1': 0.4663354754447937,
 '5LE5_C_5LE5_G_2': 0.4464556872844696,
 '5LE5_C_5LE5_G_4': 0.38718006014823914,
 '5LE5_C_5LE5_G_0': 0.5314561724662781,
 '5LE5_C_5LE5_G_3': 0.46020302176475525,
 '5LE5_C_5LE5_H_0': 0.021890360862016678,
 '5LE5_C_5LE5_H_1': 0.008743071928620338,
 '5LE5_C_5LE5_H_2': 0.0063130976632237434,
 '5LE5_C_5LE5_H_4': 0.007555801887065172,
 '5LE5_C_5LE5_H_3': 0.008787695318460464,
 '5LE5_a_5LE5_I_0': 0.1307997852563858,
 '5LE5_a_5LE5_I_2': 0.01896451972424984,
 '5LE5_a_5LE5_I_4': 0.0044169956818223,
 '5LE5_a_5LE5_I_1': 0.07931222766637802,
 '5LE5_a_5LE5_I_3': 0.005401238799095154,
 '5LE5_C_5LE5_K_1': 0.11577267199754715,
 '5LE5_C_5LE5_K_0': 0.028661809861660004,
 '5LE5_C_5LE5_K_3': 0.01235415693372488,
 '5LE5_C_5LE5_K_2': 0.06449677050113678,
 '5LE5_C_5LE5_K_4': 0.05348897725343704,
 '5LE5_D_5LE5_I_1': 0.07473459094762802,
 '5LE5_D_5LE5_I_0': 0.11803630739450455,
 '5LE5_D_5LE5_I_3': 0.020958086475729942,
 '5LE5_D_5LE5_I_2': 0.05429884418845177,
 '5LE5_D_5LE

In [75]:
convert_dic={'A':'A1', 'B':'A2', 'C':'A3', 'D':'A4', 'E':'A5', 'F':'A6', 'G':'A7', 'H':'H1', 'I':'H2', 'J':'H3', 'K':'H4', 'L':'H5', 'a':'H6', 'b':'H7'}
convert_dic2 = {v: k for k, v in convert_dic.items()}
interactions=[]
for chain in groups_dic:
    for cluster in groups_dic[chain]:
        chain1=convert_dic[chain]
        #print (chain1, cluster)
        scores=[]
        for each in cluster:
            chain2=each[:2]
            rank=each[-1]
            pair1,pair2 = '5LE5_%s_5LE5_%s_%s'%(convert_dic2[chain1],convert_dic2[chain2],rank), '5LE5_%s_5LE5_%s_%s'%(convert_dic2[chain2],convert_dic2[chain1],rank)
            if pair1 in predictions:
                scores.append(predictions[pair1])
            else:
                scores.append(predictions[pair2])
        assert len(scores)== len(cluster)
        chosen_one=cluster[scores.index(max(scores))]
        interactions.append('%s_%s'%(chain1,chosen_one))
print (interactions)

['A1_H2_1', 'A1_A4_1', 'A1_A7_1', 'A2_H3_0', 'A2_A3_1', 'A2_A1_2', 'A3_H4_1', 'A3_A2_1', 'A3_A4_0', 'A4_A3_0', 'A4_H4_1', 'A5_H6_1', 'A5_A6_0', 'A6_H6_0', 'A6_A7_0', 'A6_A5_0', 'A7_H1_1', 'A7_A1_1', 'A7_A6_0', 'H1_H5_0', 'H1_A7_1', 'H1_H2_0', 'H1_H6_0', 'H2_H5_0', 'H2_H1_0', 'H2_H4_1', 'H2_A1_1', 'H3_H3_0', 'H3_H4_0', 'H3_A2_0', 'H4_H5_0', 'H4_A3_1', 'H4_H2_1', 'H4_A4_1', 'H5_H1_0', 'H5_H6_0', 'H5_A4_0', 'H6_A5_1', 'H6_H5_0', 'H6_H1_0', 'H7_A7_0', 'H7_H7_0', 'H7_H1_0', 'H7_A5_2']


In [27]:
interactions2=[i.split('_') for i in interactions]
interactions2=[set(list(filter(lambda x: len(x)>1,i))) for i in interactions2]
interactions2 = list(set(map(frozenset, interactions2)))
interactions2

[frozenset({'A6', 'A7'}),
 frozenset({'H3'}),
 frozenset({'H1', 'H7'}),
 frozenset({'H1', 'H6'}),
 frozenset({'A3', 'H4'}),
 frozenset({'A3', 'A4'}),
 frozenset({'A5', 'H6'}),
 frozenset({'H4', 'H5'}),
 frozenset({'A1', 'A7'}),
 frozenset({'A1', 'A4'}),
 frozenset({'H2', 'H4'}),
 frozenset({'A2', 'H3'}),
 frozenset({'A1', 'H1'}),
 frozenset({'H6', 'H7'}),
 frozenset({'H5', 'H6'}),
 frozenset({'H3', 'H4'}),
 frozenset({'H7'}),
 frozenset({'A4', 'H4'}),
 frozenset({'H1', 'H2'}),
 frozenset({'A5', 'A6'}),
 frozenset({'A6', 'H6'}),
 frozenset({'A2', 'A3'}),
 frozenset({'H1', 'H5'}),
 frozenset({'A1', 'A2'}),
 frozenset({'A1', 'H2'}),
 frozenset({'A6', 'H7'}),
 frozenset({'A7', 'H1'}),
 frozenset({'H2', 'H6'})]